In [53]:
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

def find_low_pixel_indices(image_path, threshold, apply_noise_reduction=False, blur_radius=2):
    """
    画像を読み込み、ノイズ除去（オプション）、フラット化して、画素値がthresholdより小さい画素のインデックスを返す。

    Parameters:
    - image_path (str): 画像ファイルのパス。
    - threshold (int or float): 閾値。
    - apply_noise_reduction (bool): ノイズ除去を適用するかどうか。
    - blur_radius (int or float): ガウシアンブラーの半径。

    Returns:
    - indices (numpy.ndarray): 条件を満たす画素のインデックス配列。
    - original_shape (tuple): 画像の元の形状。
    - image_array (numpy.ndarray): 処理後の画像のNumPy配列。
    """
    # 画像をグレースケールで読み込む
    image = Image.open(image_path).convert('L')

    # ノイズ除去を適用
    if apply_noise_reduction:
        image = image.filter(ImageFilter.GaussianBlur(radius=blur_radius))

    image_array = np.array(image)
    original_shape = image_array.shape

    # フラット化
    flat_image = image_array.flatten()

    # 閾値より小さい画素のインデックスを取得
    indices = np.where(flat_image < threshold)[0]

    return indices, original_shape, image_array

def create_heatmap(image_array, threshold):
    """
    画像データからヒートマップを作成し、閾値より小さい画素を強調表示する。

    Parameters:
    - image_array (numpy.ndarray): 画像のNumPy配列。
    - threshold (int or float): 閾値。
    """
    # 閾値より小さい画素をマスク
    mask = image_array < threshold

    # ヒートマップの作成
    plt.figure(figsize=(8, 6))
    plt.imshow(image_array, cmap='gray', interpolation='nearest')
    plt.colorbar(label='Pixel Intensity')

    # 閾値より小さい画素を赤色で重ねる
    # マスクされた部分を赤色に変換
    red_overlay = np.zeros_like(image_array)
    red_overlay[mask] = 255  # 赤色の強度を最大に設定

    # 赤色のチャネルを追加
    plt.imshow(red_overlay, cmap='Reds', alpha=0.1)

    plt.title(f'Heatmap of Pixels Below Threshold ({threshold})')
    plt.axis('off')
    plt.show()

In [ ]:
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
image_path = DATA_PATH + "/hadamard128_cap_240814/hadamard_1.png"

threshold_value = 12

apply_noise_reduction = True
blur_radius = 4  # ガウシアンブラーの半径を設定（必要に応じて調整）

# 関数を呼び出してインデックスを取得
indices, shape, image_array = find_low_pixel_indices(
    image_path,
    threshold_value,
    apply_noise_reduction=apply_noise_reduction,
    blur_radius=blur_radius
)

print(f"Total pixels below {threshold_value}: {len(indices)}")
print(f"Indices of pixels below {threshold_value}:")
print(indices)

# ヒートマップを作成
create_heatmap(image_array, threshold_value)

In [59]:
import numpy as np

def delete_rows(matrix: np.ndarray, indices: list) -> np.ndarray:
    """
    指定したインデックスの行を削除した新しい行列を返します。

    Parameters:
    - matrix (np.ndarray): 元の行列
    - indices (list): 削除する行のインデックスのリスト

    Returns:
    - np.ndarray: 行が削除された新しい行列
    """
    return np.delete(matrix, indices, axis=0)

def revive_rows(reduced_matrix: np.ndarray, indices: list, total_rows: int) -> np.ndarray:
    """
    削除された行を0ベクトルとして復活させた行列を返します。

    Parameters:
    - reduced_matrix (np.ndarray): 行が削除された行列
    - indices (list): 復活させる行のインデックスのリスト
    - total_rows (int): 元の行列の総行数

    Returns:
    - np.ndarray: 指定した行が0ベクトルとして復活された行列
    """
    # 新しい行列を全て0で初期化
    revived_matrix = np.zeros((total_rows, reduced_matrix.shape[1]), dtype=reduced_matrix.dtype)
    
    # 削除されていない行のインデックスを計算
    remaining_indices = [i for i in range(total_rows) if i not in indices]
    
    # 残っている行を新しい行列にコピー
    revived_matrix[remaining_indices, :] = reduced_matrix
    
    return revived_matrix

In [ ]:
original_matrix = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12],
    [13, 14, 15]
])

print("元の行列:")
print(original_matrix)

indices_to_delete = [1, 3]

reduced_matrix = delete_rows(original_matrix, indices_to_delete)
print("\n行を削除した行列:")
print(reduced_matrix)

reduced_matrix *= 2
revived_matrix = revive_rows(reduced_matrix, indices_to_delete, original_matrix.shape[0])
print("\n削除した行を0ベクトルとして復活させた行列:")
print(revived_matrix)